In [ ]:
import pandas as pd

In [17]:
#read the data as string using open
with open('result.txt') as f:
    data = f.read()


In [18]:
split_data = data.split("------------------------")

In [30]:
# find the 


In [56]:
# find the value of the key, end with \n or space
import re
def find_value(data, key):
    pattern = f"{key}[0-9a-aA-Z]+[\n\s,]*"
    return re.findall(pattern, data)


In [57]:
lines = split_data[1].split("\n")

In [58]:
find_value(lines[15], "step:  ")

['step:  500 ']